## Instagram personal followers crawler & remover

Warning .. don't run all cells at once!

In [1]:
from selenium.webdriver.firefox.options import Options
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from datetime import datetime
import pandas as pd


### Initiate firefox browser

In [2]:
options = Options()
driver = webdriver.Firefox(options=options)

### Instagram user login & Crawling followers

Execute cell below and login manually on the firefox browser (the code will wait for 5 mins)

Please click "Not Now" if there's a "Save Login" page

In [3]:
HOST = 'http://www.instagram.com'
driver.get(HOST)

# try:
#     WebDriverWait(driver, 300).until(
#             EC.presence_of_element_located((By.XPATH , '//div[@class ="cmbtv"]'))
#             ).click()
# except Exception as e:
#     print("No save login prompt")
    
element = WebDriverWait(driver, 300).until(
        EC.presence_of_element_located((By.XPATH , '//div[@class ="_aacl _aaco _aacu _aacx _aada"]'))
        )
username = element.text
driver.get(urljoin(HOST,element.text+"/followers"))

div_followers = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH , '//div[@class ="_aano"]'))
        )

current = len(driver.find_elements_by_css_selector('div._ab9-'))

verical_ordinate = 100
num_followers = int(driver.find_element(By.CSS_SELECTOR,'li._aa_5:nth-child(2) > a:nth-child(1) > div:nth-child(1) > span:nth-child(1)').text)
print("Num of followers :"+str(num_followers)+", first fetch :"+str(current))
while current < num_followers :
    current = len(driver.find_elements_by_css_selector('div._ab9-'))
    print("Now collecting followers :"+str(current),end='\r')
    driver.execute_script("arguments[0].scrollTop = arguments[1]", div_followers, verical_ordinate)
    verical_ordinate +=5000
    time.sleep(2)

C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\2649968052.py:21: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  current = len(driver.find_elements_by_css_selector('div._ab9-'))


Num of followers :562, first fetch :10


C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\2649968052.py:27: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  current = len(driver.find_elements_by_css_selector('div._ab9-'))


### Dump mutual & non mutual followers to CSV

In [4]:
import pandas as pd

username_mutual = []
fullname_mutual = []
div_id_mutual = []

username_not_mutual = []
fullname_not_mutual = []
div_id_not_mutual = []

for i in driver.find_elements_by_css_selector('div._ab9-'):
    div = i.find_elements_by_css_selector('div._ab94')
    username = i.find_elements_by_css_selector('div._ab94')[3].text
    full_name = i.find_elements_by_css_selector('div._ab94')[4].text
    id = i.get_attribute("aria-labelledby").split(" ")[0]
    
    #mutual
    if '·' not in username and '·' not in full_name:
        username_mutual.append(username)
        fullname_mutual.append(full_name)
        div_id_mutual.append(id)
    
    #not mutual    
    else:
        full_name = i.find_elements_by_css_selector('div._ab94')[6].text
        username = username.split('\n')[0]
        username_not_mutual.append(username)
        fullname_not_mutual.append(full_name)
        div_id_not_mutual.append(id)

df_mutual = pd.DataFrame({
    "username": username_mutual,
    "fullname" : fullname_mutual,
    "id" : div_id_mutual
})

df_non_mutual =  pd.DataFrame({
    "username": username_not_mutual,
    "fullname" : fullname_not_mutual,
    "id" : div_id_not_mutual
})

now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

df_mutual.to_csv("list_mutual_followers_"+now+".csv")
df_non_mutual.to_csv("list_non_mutual_followers_"+now+".csv")


C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\1319165261.py:11: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  for i in driver.find_elements_by_css_selector('div._ab9-'):
C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\1319165261.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  div = i.find_elements_by_css_selector('div._ab94')
C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\1319165261.py:13: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  username = i.find_elements_by_css_selector('div._ab94')[3].text
C:\Users\johantandy\AppData\Local\Temp\ipykernel_8144\1319165261.py:14: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=c

### Remove top x non mutual followers

In [19]:
x_followers = 10
idx = 0 
now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
with open("list_removed_non_mutual_followers"+now+".csv", "w") as file_object:
    file_object.write("username,fullname\n")
for idx,fol in enumerate(div_id_not_mutual):
    if idx < x_followers:
        full_name = fullname_not_mutual[idx]
        username = username_not_mutual[idx]
#         print(username,full_name)
        with open("list_removed_non_mutual_followers"+now+".csv", "a", encoding="utf-8") as file_object:
            file_object.write(','.join([username,full_name])+"\n")

        driver.find_element(By.XPATH, '//div[@id ="'+fol+'"]').click()

        #uncomment this if u ready to remove your followers
#             element = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH , '//button[@class ="_a9-- _a9-_"]'))
#             )
        #########################################

        #run this if you want to check which followers will be removed
        # comment this if you want to remove your followers
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH , '//button[@class ="_a9-- _a9_1"]'))
        )
        #########################################


        element.click()
        print("follower removed : "+str(idx+1),end='\r')
    else:
        break
        

In [16]:
len(username_mutual)+len(username_not_mutual)

562